In [1]:
import os
import pandas as pd
import numpy as np 

In [2]:
os.getcwd()

'/Users/kinshingwong/Documents/GitHub/PyEmits/examples'

In [3]:
os.chdir('/Users/kinshingwong/Documents/GitHub/PyEmits')

In [4]:
from pyemits.core.ml.regression.trainer import RegTrainer, RegressionDataModel, SliceableDeque, ParallelRegTrainer, KFoldCVTrainer, MultiOutputRegTrainer
from pyemits.core.ml.regression.predictor import RegPredictor
from pyemits.core.ml.regression.nn import KerasWrapper
from pyemits.core.preprocessing.splitting import extract_tensor_data, SlidingWindowSplitter
from pyemits.core.preprocessing.scaling import ForecastArrayScaling
from pyemits.core.ml.base import save_model, load_model

2021-10-21 20:06:27.964623: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from pyemits.common.stats import cal_reg_metrics, RegressionMetrics
from pyemits.core.preprocessing.scaling import ForecastArrayScaling
from pyemits.common.utils.misc_utils import slice_iterables
from pyemits.core.preprocessing.misc_utils import create_time_features, df_create_time_features

In [6]:
import numpy as np

from pyemits.core.ml.regression.trainer import RegTrainer, RegressionDataModel
from pyemits.core.ml.regression.nn import KerasWrapper
from pyemits.common.config_model import KerasSequentialConfig

X = np.random.randint(1, 100, size=(1000, 10, 10))
y = np.random.randint(1, 100, size=(1000, 4))

from keras.layers import Dense, Dropout, LSTM
from keras import Sequential

keras_lstm_model = KerasWrapper(nickname='LSTM')
config = KerasSequentialConfig(layer=[LSTM(128,
                                           activation='softmax',
                                           input_shape=(10, 10),
                                           ),
                                      Dropout(0.1),
                                      Dense(4)],
                               compile=dict(loss='mse', optimizer='adam', metrics=['mse']))

raw_data_model = RegressionDataModel(X, y)
trainer = RegTrainer([keras_lstm_model],
                     [config],
                     raw_data_model, 
                     {'fit_config' : [dict(epochs=10, batch_size=32)]})
trainer.fit()

2021-10-21 20:06:28.213592: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
32/32 [==============================] - 1s 6ms/step - loss: 3348.0610 - mse: 3348.0610
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 3339.5229 - mse: 3339.5229
Epoch 3/10
32/32 [==============================] - 0s 5ms/step - loss: 3332.2429 - mse: 3332.2429
Epoch 4/10
32/32 [==============================] - 0s 6ms/step - loss: 3325.7871 - mse: 3325.7871
Epoch 5/10
32/32 [==============================] - 0s 6ms/step - loss: 3319.0457 - mse: 3319.0457
Epoch 6/10
32/32 [==============================] - 0s 7ms/step - loss: 3312.1838 - mse: 3312.1838
Epoch 7/10
32/32 [==============================] - 0s 7ms/step - loss: 3305.8027 - mse: 3305.8027
Epoch 8/10
32/32 [==============================] - 0s 7ms/step - loss: 3299.3516 - mse: 3299.3516
Epoch 9/10
32/32 [==============================] - 0s 7ms/step - loss: 3293.0249 - mse: 3293.0249
Epoch 10/10
32/32 [==============================] - 0s 7ms/step - loss: 3286.7917 - mse: 3286.7917


# Keras Wrapper

In [6]:
df = pd.read_parquet('/Users/kinshingwong/Documents/GitHub/cam-Analytics/dataAnalyticsRework/test_data/20201123 Five Sites Data/ChillerPlant_TG_OG.parquet')
df['weekday'] = df.index.weekday

In [7]:
df['Qev'] = df['mw']*4.186*(df['Tchwr']-df['Tchws'])

In [9]:
# df['year'] = create_time_features(df.index.to_numpy(),['year'])[0]
# df['month'] = create_time_features(df.index.to_numpy(),['month'])[0]
# df['day'] = create_time_features(df.index.to_numpy(),['day'])[0]
# df['hour'] = create_time_features(df.index.to_numpy(),['hour'])[0]
# df['minutes'] = create_time_features(df.index.to_numpy(),['minutes'])[0]
df = df_create_time_features(df)


In [52]:
groupby_df = df.groupby(['year','month','day','hour'], as_index=False).mean()

In [53]:
# not a strict methodology for nan handling, it will make LSTM have bad performance
# but it just for fast demonstrating on Keras Wrapper function, performance not in my consideration
groupby_df = groupby_df.fillna(method='ffill')

In [45]:
splitter = SlidingWindowSplitter(24,24)

In [54]:
X, y = splitter.split(groupby_df[['Tout','Tout']].to_numpy(), groupby_df['Tout'].to_numpy())

In [55]:
X = np.array(X)
y = np.array(y)

In [32]:
keras_wrapper = KerasWrapper.from_simple_lstm_model((24,2), 24)

2021-10-21 13:59:53.497213: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [65]:
trainer = RegTrainer([keras_wrapper],[None],RegressionDataModel(X,y), {'fit_config':[dict(epochs=20,batch_size=32, validation_split=0.2)]})


In [66]:
trainer.fit()

Epoch 1/20
978/978 [==============================] - 12s 12ms/step - loss: 509.8878 - mse: 509.8878 - val_loss: 476.4556 - val_mse: 476.4556
Epoch 2/20
978/978 [==============================] - 11s 11ms/step - loss: 432.8757 - mse: 432.8757 - val_loss: 402.4171 - val_mse: 402.4171
Epoch 3/20
978/978 [==============================] - 11s 12ms/step - loss: 363.7594 - mse: 363.7594 - val_loss: 336.0959 - val_mse: 336.0959
Epoch 4/20
978/978 [==============================] - 11s 12ms/step - loss: 303.4343 - mse: 303.4343 - val_loss: 277.1541 - val_mse: 277.1541
Epoch 5/20
978/978 [==============================] - 11s 11ms/step - loss: 249.1144 - mse: 249.1144 - val_loss: 225.1037 - val_mse: 225.1037
Epoch 6/20
978/978 [==============================] - 13s 13ms/step - loss: 201.9685 - mse: 201.9685 - val_loss: 179.7179 - val_mse: 179.7179
Epoch 7/20
978/978 [==============================] - 13s 14ms/step - loss: 162.3285 - mse: 162.3285 - val_loss: 140.9066 - val_mse: 140.9066
Epoch 

In [67]:
predictor = RegPredictor(trainer.clf_models, 'RegTrainer')

In [71]:
predictor.predict(RegressionDataModel(X[-100:-95]))

SliceableDeque([array([[24.67835 , 24.663052, 24.666267, 24.660236, 24.6636  , 24.660267,
                        24.671782, 24.657413, 24.66935 , 24.661285, 24.655651, 24.664669,
                        24.650373, 24.650177, 24.65356 , 24.653622, 24.639584, 24.650993,
                        24.65087 , 24.659151, 24.66613 , 24.66764 , 24.672745, 24.663929],
                       [24.386086, 24.370455, 24.373924, 24.36892 , 24.372303, 24.370428,
                        24.383343, 24.36659 , 24.382332, 24.373465, 24.36597 , 24.378094,
                        24.360897, 24.361399, 24.364697, 24.365307, 24.34899 , 24.362242,
                        24.362844, 24.370787, 24.378794, 24.379452, 24.385101, 24.373447],
                       [24.120424, 24.10404 , 24.107998, 24.103556, 24.107367, 24.106346,
                        24.120699, 24.102062, 24.12062 , 24.11095 , 24.102224, 24.11656 ,
                        24.097101, 24.098137, 24.101667, 24.102665, 24.08451 , 24.09934 ,
        

In [74]:
r = predictor.predict(RegressionDataModel(X))[0]

In [77]:
rr = pd.DataFrame({'prediction':r.ravel(), 'original': y.ravel()})